# PyTorch on GPU: first steps

### Put tensor to GPU

In [1]:
import torch

device = torch.device("cuda:0")

In [2]:
my_tensor = torch.Tensor([1., 2., 3., 4., 5.])

mytensor = my_tensor.to(device)

In [3]:
mytensor

tensor([1., 2., 3., 4., 5.], device='cuda:0')

In [4]:
my_tensor

tensor([1., 2., 3., 4., 5.])

### Put model to GPU

In [5]:
from torch import nn

class Model(nn.Module):
    def __init__(self, input_size, output_size):
        super(Model, self).__init__()
        self.fc = nn.Linear(input_size, output_size)

    def forward(self, input):
        output = self.fc(input)
        print("\tIn Model: input size", input.size(),
              "output size", output.size())

        return output

In [6]:
input_size = 128
output_size = 128
model = Model(input_size, output_size)

In [7]:
device = torch.device("cuda:0")
model.to(device)

Model(
  (fc): Linear(in_features=128, out_features=128, bias=True)
)

### Data parallelism

In [8]:
from torch.nn import DataParallel

In [9]:
torch.cuda.is_available()

True

In [10]:
torch.cuda.device_count()

2

### Part on CPU, part on GPU

In [12]:
device = torch.device("cuda:0")

class Model(nn.Module):

    def __init__(self, input_size, output_size):
        super(Model, self).__init__()
        self.fc = nn.Linear(input_size, 100)       
        self.fc2 = nn.Linear(100, output_size).to(device)

    def forward(self, x):
        # Compute first layer on CPU
        x = self.fc(x)

        # Transfer to GPU
        x = x.to(device)

        # Compute second layer on GPU
        x = self.fc2(x)
        return x

In [13]:
input_size = 100
output_size = 50

data_length = 1000

In [14]:
data = torch.randn(data_length, input_size)

In [15]:
model = Model(input_size, output_size)
model.forward(data)

tensor([[-0.2434, -0.4078,  0.2789,  ...,  0.2297,  0.2767,  0.7111],
        [-0.0210, -0.4034,  0.1874,  ..., -0.2352, -0.6421, -0.8545],
        [ 0.6456,  0.0599, -0.1457,  ..., -0.1813, -0.6686,  0.3703],
        ...,
        [-0.2104,  0.1604,  0.1981,  ..., -0.2063,  0.1795, -0.2136],
        [-0.2140, -0.0474, -0.0815,  ...,  0.0959, -0.1747,  0.1745],
        [-0.1903, -0.3247,  0.0406,  ...,  0.3202, -0.5894,  0.3529]],
       device='cuda:0', grad_fn=<ThAddmmBackward>)